In [1]:
# Import necessary modules and functions

import pandas as pd
import seaborn as sns
import numpy as np
import nltk
import spacy
import datetime

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 16, 8

from scipy.stats import norm
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

kaggledf = pd.read_csv('KS_train_data.csv', delimiter = ',')
kaggletestdf = pd.read_csv('KS_test_data.csv', delimiter = ';')

In [2]:
###url extraction for training data
pd.set_option('display.max_colwidth', None)
#borrowed Nick's efficient method
url_split = []
for url in kaggledf['project_url']:
       url = url.split('/')[-1]
       url_split.append(url.replace('-',' '))

kaggledf['project_url_split'] = url_split
cols=['project_url','project_url_split']
kaggledf[cols]


,project_url,project_url_split
0,https://www.kickstarter.com/projects/hilaryemersonlay/elsewhere-studios-artist-in-residency-program,elsewhere studios artist in residency program
1,https://www.kickstarter.com/projects/17146650/1000-artists-presidential-inauguration-2013,1000 artists presidential inauguration 2013
2,https://www.kickstarter.com/projects/belveal/myphonehenge,myphonehenge
3,https://www.kickstarter.com/projects/1078130915/walk-through,walk through
4,https://www.kickstarter.com/projects/1044092769/a-kitchen-for-habitable-spaces,a kitchen for habitable spaces
...,...,...
99995,https://www.kickstarter.com/projects/jimmycarbone/help-fund-the-back-room-arts-space-at-jimmys-no-43,help fund the back room arts space at jimmys no 43
99996,https://www.kickstarter.com/projects/rvacomedy/the-coalition-theater-0,the coalition theater 0
99997,https://www.kickstarter.com/projects/1601712092/the-voix-de-ville-a-pop-up-theater-and-cabaret,the voix de ville a pop up theater and cabaret
99998,https://www.kickstarter.com/projects/967829854/save-the-studio,save the studio


In [3]:
###url extraction for test data

pd.set_option('display.max_colwidth', None)
#borrowed Nick's efficient method
url_split_test = []
for url in kaggletestdf['project_url']:
       url = url.split('/')[-1]
       url_split_test.append(url.replace('-',' '))

kaggletestdf['project_url_split'] = url_split_test
cols=['project_url','project_url_split']
kaggletestdf[cols]


,project_url,project_url_split
0,https://www.kickstarter.com/projects/1536850207/martin-luther-king-jr-sculpture-on-campus,martin luther king jr sculpture on campus
1,https://www.kickstarter.com/projects/2039713907/emergence-surreal-oil-paintings-by-jj-long,emergence surreal oil paintings by jj long
2,https://www.kickstarter.com/projects/1442314751/pmaftw,pmaftw
3,https://www.kickstarter.com/projects/199636585/metropolove-book-project-to-be-completed-from-2010,metropolove book project to be completed from 2010
4,https://www.kickstarter.com/projects/jimgroom/ds106-the-open-online-community-of-digital-storyte,ds106 the open online community of digital storyte
...,...,...
78060,https://www.kickstarter.com/projects/552677239/improvmania-improv-comedy-theater-chandler-arizona,improvmania improv comedy theater chandler arizona
78061,https://www.kickstarter.com/projects/1947692763/build-the-house-of-dads,build the house of dads
78062,https://www.kickstarter.com/projects/1365853422/the-pocket-theater-seattle-needs-a-place-to-perfor,the pocket theater seattle needs a place to perfor
78063,https://www.kickstarter.com/projects/schmeater/shelter-the-schmee,shelter the schmee


In [4]:
####Tf/IDF for url extractions for the top 100 words (can change this later)
vect = TfidfVectorizer(analyzer='word', stop_words='english', max_features = 100)
#perform tf/idf on training data
X = vect.fit_transform(kaggledf['project_url_split'])
#convert into a dataframe
X_df = pd.DataFrame(data = X.A, columns=vect.get_feature_names(), index=kaggledf.index)
#perform tf/idf on test data
X_test = vect.transform(kaggletestdf['project_url_split'])
#convert into a dataframe
X_testdf = pd.DataFrame(data = X_test.A, columns=vect.get_feature_names(), index=kaggletestdf.index)
pd.set_option('display.max_columns', None)
#add the new columns/features to training and test datasets
kaggledf = kaggledf.join(X_df)
kaggletestdf = kaggletestdf.join(X_testdf)
kaggledf


,project_id,backers_count,blurb,category,converted_pledged_amount,country,created_at,currency,deadline,fx_rate,goal,launched_at,name,pledged,staff_pick,usd_pledged,location,funded,subcategory,project_url,reward_url,project_url_split,100,2015,3d,adventure,album,american,animated,app,art,artist,big,black,book,bring,calendar,card,cards,cd,childrens,city,coffee,collection,comedy,comic,community,company,dance,day,debut,design,documentary,dream,edition,ep,fantasy,farm,feature,festival,film,food,free,fund,game,graphic,hand,help,high,home,horror,house,issue,journey,kids,la,length,lets,life,light,little,live,love,magazine,make,making,man,mobile,movie,music,musical,new,novel,open,original,photo,photography,play,playing,presents,print,production,project,record,recording,release,season,series,short,smart,space,stories,story,studio,time,tour,truck,video,volume,web,world,worlds
0,KS_000000,80,"I will be an artist-in-residence at Elsewhere Studios this summer, where I will stretch my wings and see where I land!",art,3596,US,1325980505,USD,1334866560,1.000000,2800.0,1332969260,Elsewhere Studios artist-in-residency program!,3596.00,False,3596.000000,"Paonia, CO",True,mixed media,https://www.kickstarter.com/projects/hilaryemersonlay/elsewhere-studios-artist-in-residency-program,https://www.kickstarter.com/projects/hilaryemersonlay/elsewhere-studios-artist-in-residency-program/rewards,elsewhere studios artist in residency program,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,KS_000001,82,1000 Artists is a public art-making installation on the Washington Mall which will occur on January 21 and 22nd 2013.,art,4586,US,1330926084,USD,1336440145,1.000000,4500.0,1332984145,1000 Artists: Presidential Inauguration 2013,4586.00,False,4586.000000,"Washington, DC",True,public art,https://www.kickstarter.com/projects/17146650/1000-artists-presidential-inauguration-2013,https://www.kickstarter.com/projects/17146650/1000-artists-presidential-inauguration-2013/rewards,1000 artists presidential inauguration 2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,KS_000002,21,"The Sequel to ‘My Favorite Machine”, ""MyPhoneHenge"" celebrates Community across Virtual and Real Worlds.",art,5217,US,1332382894,USD,1338175739,1.000000,5000.0,1332991739,MyPhoneHenge,5217.00,False,5217.000000,"Frisco, TX",True,sculpture,https://www.kickstarter.com/projects/belveal/myphonehenge,https://www.kickstarter.com/projects/belveal/myphonehenge/rewards,myphonehenge,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,KS_000003,37,A film exploring the role and value of art education today through the history of the seminal California Institute of the Arts.,art,7160,GB,1332342451,USD,1334806096,1.000000,6500.0,1332991696,Walk-Through,7160.00,False,7160.000000,"Glasgow, UK",True,art,https://www.kickstarter.com/projects/1078130915/walk-through,https://www.kickstarter.com/projects/1078130915/walk-through/rewa

In [5]:
####Tf/IDF for url extractions for the top 100 words (can change this later)
vect_blurb = TfidfVectorizer(analyzer='word', stop_words='english', max_features = 100)
#convert blurbs into unicode data type
Xblurb = kaggledf['blurb'].values.astype('U')
Xblurb_test = kaggletestdf['blurb'].values.astype('U')
#perform td/idf on training data
Xblurb = vect_blurb.fit_transform(Xblurb)
#convert into a dataframe
Xblurb_df = pd.DataFrame(data = Xblurb.A, columns=vect_blurb.get_feature_names(), index=kaggledf.index)
#perform tf/idf on test data
Xblurb_test = vect_blurb.transform(Xblurb_test)
#convert into a dataframe
Xblurb_testdf = pd.DataFrame(data = Xblurb_test.A, columns=vect_blurb.get_feature_names(), index=kaggletestdf.index)
pd.set_option('display.max_columns', None)
vect_blurb.get_feature_names()
#find the different column names (or words) so there are no duplicate columns between the words selected 
#from the url and the blurbs
cols_to_use = Xblurb_df.columns.difference(kaggledf.columns)
#add the new columns/features to training and test datasets
#this ends up adding 44 new columns instead of 100 since there were a lot of overlapping words from the url data
kaggledf = kaggledf.join(Xblurb_df[cols_to_use])
kaggletestdf = kaggletestdf.join(Xblurb_testdf[cols_to_use])
kaggledf

,project_id,backers_count,blurb,category,converted_pledged_amount,country,created_at,currency,deadline,fx_rate,goal,launched_at,name,pledged,staff_pick,usd_pledged,location,funded,subcategory,project_url,reward_url,project_url_split,100,2015,3d,adventure,album,american,animated,app,art,artist,big,black,book,bring,calendar,card,cards,cd,childrens,city,coffee,collection,comedy,comic,community,company,dance,day,debut,design,documentary,dream,edition,ep,fantasy,farm,feature,festival,film,food,free,fund,game,graphic,hand,help,high,home,horror,house,issue,journey,kids,la,length,lets,life,light,little,live,love,magazine,make,making,man,mobile,movie,music,musical,new,novel,open,original,photo,photography,play,playing,presents,print,production,project,record,recording,release,season,series,short,smart,space,stories,story,studio,time,tour,truck,video,volume,web,world,worlds,artists,band,based,beautiful,best,build,children,create,creating,designed,experience,family,featuring,friends,fun,great,inspired,just,like,local,looking,money,need,needs,old,people,quality,real,set,share,small,songs,support,unique,use,using,ve,want,way,women,work,year,years,young
0,KS_000000,80,"I will be an artist-in-residence at Elsewhere Studios this summer, where I will stretch my wings and see where I land!",art,3596,US,1325980505,USD,1334866560,1.000000,2800.0,1332969260,Elsewhere Studios artist-in-residency program!,3596.00,False,3596.000000,"Paonia, CO",True,mixed media,https://www.kickstarter.com/projects/hilaryemersonlay/elsewhere-studios-artist-in-residency-program,https://www.kickstarter.com/projects/hilaryemersonlay/elsewhere-studios-artist-in-residency-program/rewards,elsewhere studios artist in residency program,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,KS_000001,82,1000 Artists is a public art-making installation on the Washington Mall which will occur on January 21 and 22nd 2013.,art,4586,US,1330926084,USD,1336440145,1.000000,4500.0,1332984145,1000 Artists: Presidential Inauguration 2013,4586.00,False,4586.000000,"Washington, DC",True,public art,https://www.kickstarter.com/projects/17146650/1000-artists-presidential-inauguration-2013,https://www.kickstarter.com/projects/17146650/1000-artists-presidential-inauguration-2013/rewards,1000 artists presidential inauguration 2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.630263,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,KS_000002,21,"The Sequel to ‘My Favorite Machine”, ""MyPhoneHenge"" celebrates Community across Virtual and Real Worlds.",art,5217,US,1332382894,USD,1338175739,1.000000,5000.0,1332991739,MyPhoneHenge,5217.00,False,5217.000000,"Frisco, TX",True,sculpture,https://www.kickstarter.com/projects/belveal/myphonehenge,https://www.kickstarter.com/projects/belveal/myphonehenge/rewards,myphonehenge,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
#nltk.download('vader_lexicon', quiet=False) # you can download 1 time then comment out
from nltk.sentiment.vader import SentimentIntensityAnalyzer #verify download
vader_model = SentimentIntensityAnalyzer()

In [ ]:
nlp = spacy.load('en_core_web_sm') # en_core_web_sm

In [ ]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)
#categorical ranges for goal and pledged, sentiment vader (numerical & categorical)  
#columns to be removed: url's, location, id,
kaggledf.dropna(inplace=True)
kaggledf

In [ ]:
def getSeconds(row):
    t1 = pd.Timestamp(datetime.datetime.fromtimestamp(row["created_at"]).isoformat())
    t2 = pd.Timestamp(datetime.datetime.fromtimestamp(row["deadline"]).isoformat())
    difference = t2-t1
    
    return difference.days * 86400 + difference.seconds
    
kaggle_df["seconds"] = kaggle_df.apply (lambda row: getSeconds(row), axis=1)



In [ ]:
pd.set_option("display.max_rows", 10, "display.max_columns", None)
#categorical ranges for goal and pledged, sentiment vader (numerical & categorical)  
#columns to be removed: url's, location, id,
kaggle_df

In [ ]:
def run_vader(textual_unit, 
              lemmatize=False, 
              parts_of_speech_to_consider=set(),
              verbose=0):
    """
    Run VADER on a sentence from spacy
    
    :param str textual unit: a textual unit, e.g., sentence, sentences (one string)
    (by looping over doc.sents)
    :param bool lemmatize: If True, provide lemmas to VADER instead of words
    :param set parts_of_speech_to_consider:
    -empty set -> all parts of speech are provided
    -non-empty set: only these parts of speech are considered
    :param int verbose: if set to 1, information is printed
    about input and output
    
    :rtype: dict
    :return: vader output dict
    """
    doc = nlp(textual_unit)
        
    input_to_vader = []

    for sent in doc.sents:
        for token in sent:

            to_add = token.text

            if lemmatize:
                to_add = token.lemma_

                if to_add == '-PRON-': 
                    to_add = token.text

            if parts_of_speech_to_consider:
                if token.pos_ in parts_of_speech_to_consider:
                    input_to_vader.append(to_add) 
            else:
                input_to_vader.append(to_add)

    scores = vader_model.polarity_scores(' '.join(input_to_vader))
    
    if verbose >= 1:
        print()
        print('INPUT SENTENCE', sent)
        print('INPUT TO VADER', input_to_vader)
        print('VADER OUTPUT', scores)

    return scores

In [ ]:
sentences = ["Here are my sentences.",
             "It's a nice day.",
             "It's a rainy day."]

for sent in sentences:
    scores = vader_model.polarity_scores(sent)
    print()
    print('INPUT SENTENCE', sent)
    print('VADER OUTPUT', scores)

In [ ]:
run_vader(sentences[2], lemmatize=True)

In [ ]:
sent_input = kaggledf['blurb'][5]
print(sent_input)

In [ ]:
run_vader(kaggledf['blurb'][56], lemmatize=True)


In [ ]:
kaggledf['blurb']

In [ ]:
sentiment_scores = []
for sentence in kaggledf['blurb']:
    sentiment_scores.append(run_vader(sentence, lemmatize=True)['compound'])

In [ ]:
kaggledf['blurb_sentiment_scores'] = sentiment_scores

In [ ]:
kaggledf['blurb_sentiment_scores']